In [6]:
import sys
import os

# To find local version of the library
sys.path.append(os.path.join("./comp642-project/FRaverta-Notebooks"))

# here is your import
from featurizer import *

2024-04-22 21:49:55 [info     ] Downloading ZIP file with the data set...
2024-04-22 21:50:15 [info     ] Opening ZIP file...           
2024-04-22 21:50:15 [info     ] Reading CSV file...            fname=train.csv
2024-04-22 21:50:24 [info     ] Data set loaded successfully. 
2024-04-22 21:50:26 [info     ] Baseline model computed.       test_mae=5.8499749700259205 train_mae=6.481303604250604


In [7]:
data = TradingAtTheCloseDS()
train_data, test_data = data.get_train_test_data()
#train_mae, test_mae = data.compute_baseline_model()

2024-04-22 21:50:28 [info     ] Downloading ZIP file with the data set...
2024-04-22 21:50:41 [info     ] Opening ZIP file...           
2024-04-22 21:50:41 [info     ] Reading CSV file...            fname=train.csv
2024-04-22 21:50:52 [info     ] Data set loaded successfully. 


In [10]:
#drop rows with nan in wap column
train_data = train_data.dropna(subset=['wap'])
test_data = test_data.dropna(subset=['wap'])

x_train, y_train = train_data[data.categorical_features + data.numerical_features], train_data[data.y_column]
x_test, y_test = test_data[data.categorical_features + data.numerical_features], test_data[data.y_column]

# Random Forest on DATASET with all features, features nan set to 0

In [11]:
x_train.info()

# get information about nan values in each column
x_train.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4621815 entries, 0 to 4621979
Data columns (total 14 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   stock_id                 int64  
 1   imbalance_buy_sell_flag  int64  
 2   seconds_in_bucket        int64  
 3   imbalance_size           float64
 4   reference_price          float64
 5   matched_size             float64
 6   far_price                float64
 7   near_price               float64
 8   bid_price                float64
 9   bid_size                 float64
 10  ask_price                float64
 11  ask_size                 float64
 12  wap                      float64
 13  target                   float64
dtypes: float64(11), int64(3)
memory usage: 528.9 MB


stock_id                         0
imbalance_buy_sell_flag          0
seconds_in_bucket                0
imbalance_size                   0
reference_price                  0
matched_size                     0
far_price                  2553470
near_price                 2520990
bid_price                        0
bid_size                         0
ask_price                        0
ask_size                         0
wap                              0
target                           0
dtype: int64

In [12]:
x_test.info()

# get information about nan values in each column
x_test.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 615945 entries, 4621980 to 5237979
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   stock_id                 615945 non-null  int64  
 1   imbalance_buy_sell_flag  615945 non-null  int64  
 2   seconds_in_bucket        615945 non-null  int64  
 3   imbalance_size           615945 non-null  float64
 4   reference_price          615945 non-null  float64
 5   matched_size             615945 non-null  float64
 6   far_price                275293 non-null  float64
 7   near_price               279975 non-null  float64
 8   bid_price                615945 non-null  float64
 9   bid_size                 615945 non-null  float64
 10  ask_price                615945 non-null  float64
 11  ask_size                 615945 non-null  float64
 12  wap                      615945 non-null  float64
 13  target                   615945 non-null  float64
dt

stock_id                        0
imbalance_buy_sell_flag         0
seconds_in_bucket               0
imbalance_size                  0
reference_price                 0
matched_size                    0
far_price                  340652
near_price                 335970
bid_price                       0
bid_size                        0
ask_price                       0
ask_size                        0
wap                             0
target                          0
dtype: int64

In [13]:
x_train.loc[:,'far_price'] = x_train['far_price'].fillna(0)
x_train.loc[:,'near_price'] = x_train['near_price'].fillna(0)

x_test.loc[:,'far_price'] = x_test['far_price'].fillna(0)
x_test.loc[:,'near_price'] = x_test['near_price'].fillna(0)


/tmp/ipykernel_2628258/1991184684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train.loc[:,'far_price'] = x_train['far_price'].fillna(0)
/tmp/ipykernel_2628258/1991184684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train.loc[:,'near_price'] = x_train['near_price'].fillna(0)
/tmp/ipykernel_2628258/1991184684.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Create a "do-nothing" transformer
no_op_transformer = FunctionTransformer(lambda x: x)



# Numerical pipeline with feature addition
numerical_pipeline = Pipeline([
    ('do_nothing', no_op_transformer)
])

# Categorical pipeline
categorical_pipeline = Pipeline([
    ('onehot', OneHotEncoder())
])

# Combine pipelines
preprocessing_pipeline = ColumnTransformer([
    ('num', numerical_pipeline, data.numerical_features),
    ('cat', categorical_pipeline, data.categorical_features)
])

# Applying the pipeline
x_train_transformed = preprocessing_pipeline.fit_transform(x_train)
x_test_transformed = preprocessing_pipeline.transform(x_test)


random_forest = RandomForestRegressor(n_estimators=11, random_state=42, verbose=1, n_jobs=-1,  max_depth=10)
random_forest.fit(x_train_transformed, y_train.values.reshape(-1))
y_pred = random_forest.predict(x_test_transformed)

print(f"Random Forest MAE: {mean_absolute_error(y_test, y_pred)}")


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  11 | elapsed:  7.9min remaining:  3.0min
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed: 12.0min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


Random Forest MAE: 0.003943358133017773


[Parallel(n_jobs=8)]: Done   8 out of  11 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=8)]: Done  11 out of  11 | elapsed:    0.2s finished


# 2 Random forest models, one first 5 minutes other second 5


In [15]:
random_forest_first_5min = RandomForestRegressor(n_estimators=10, random_state=42, verbose=1, n_jobs=-1,  max_depth=10)
random_forest_second_5min = RandomForestRegressor(n_estimators=10, random_state=42, verbose=1, n_jobs=-1,  max_depth=10)

# select those rows which 'far_price' is not nan
x_train_transformed_first_5min = x_train_transformed[x_train_transformed['far_price'].notna()]
x_train_transformed_second_5min = x_train_transformed[x_train_transformed['far_price'].isna()]

y_train_first_5min  = y_train[x_train_transformed['far_price'].notna()]
y_train_second_5min = y_train[x_train_transformed['far_price'].isna()]

# Fit the first model
random_forest_first_5min.fit(x_train_transformed_first_5min, y_train_first_5min.values.reshape(-1))
random_forest_second_5min.fit(x_train_transformed_second_5min, y_train_second_5min.values.reshape(-1))

# Custom predict method to use different models based on the 'far_price' condition
def custom_predict(x_test):
    # Select the rows where 'far_price' is not NaN
    x_test_first_5min = x_test[x_test['far_price'].notna()]
    x_test_second_5min = x_test[x_test['far_price'].isna()]

    # Make predictions with the corresponding model
    y_pred_first_5min = random_forest_first_5min.predict(x_test_first_5min)
    y_pred_second_5min = random_forest_second_5min.predict(x_test_second_5min)

    # Create a combined prediction array
    y_pred = pd.Series(index=x_test.index)  # Initialize with NaNs

    # Assign predictions to the correct rows
    y_pred.loc[x_test_first_5min.index] = y_pred_first_5min
    y_pred.loc[x_test_second_5min.index] = y_pred_second_5min

    return y_pred


y_pred = custom_predict(x_test_transformed)
print(f"Random Forest MAE: {mean_absolute_error(y_test, y_pred)}")


IndexError: Index dimension must be 1 or 2